# <span style="color:#6C5CE7">DBRA24 Digital Twin Anomaly — End-to-End Pipeline with A1–A10 Ablations</span>

<span style="background:#E3F2FD;color:#0D47A1;padding:6px 10px;border-radius:6px;display:inline-block;">
<strong>Dataset</strong>: <code>/kaggle/input/driver-behavior-and-route-anomaly-dataset-dbra24/driver_behavior_route_anomaly_dataset_with_derived_features.csv</code>
</span>

This notebook is **ready to run on Kaggle**. It:
- trains **A1–A10 ablations** (GBM baseline, Bi-LSTM variants, TCN-Small)
- saves models to <code>/kaggle/working/models/</code>
- exports a single **Excel** report to <code>/kaggle/working/DBRA24_results.xlsx</code>
- writes **plots** (PR curves, reliability, confusion matrices, ROC) to <code>/kaggle/working/plots/</code>
- prints a **final comparison table** (easy to paste in your paper/report)

> Tip: Turn on GPU in the Kaggle Notebook settings for faster training (optional).

## <span style="color:#00B894">Table of Contents</span>
1. 🔧 Setup (Installs & Imports)  
2. ⚙️ Config (paths, seeds, hyperparameters)  
3. 📥 Load & Inspect Data  
4. 🧭 Feature Groups & Light Engineering  
5. 🧱 Windowing into Sequences (T=60s, stride=5s)  
6. ✂️ Split by Driver (train/val/test)  
7. 📏 Scaling & PyTorch Datasets  
8. 🧠 Models (Bi-LSTM, TCN-Small, Gating)  
9. 📐 Losses & Metrics (PR-AUC, F1, RMSE, ECE, TTD, FP/h, Latency, FLOPs)  
10. 🏃 Training/Evaluation Utilities  
11. 🌳 A1 — GBM (No Mobility, Temporal-Agnostic)  
12. 🧪 A2–A10 — Ablations (one-by-one)  
13. 📊 Results Export (Excel, CSV, Plots) + Final Comparison Table  
14. 📝 Notes & Tuning

## 1) 🔧 Setup — Installs
Safe to re-run. Kaggle usually has most deps. We install a few extras (quietly).

In [1]:
# If internet is restricted, you can comment these lines. On Kaggle, this is OK to run.
import sys, subprocess, importlib

def ensure(pkg, pip_name=None):
    pip_name = pip_name or pkg
    try:
        importlib.import_module(pkg)
        print(f"{pkg} ✓")
    except Exception:
        print(f"Installing {pip_name} …")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pip_name])

for p in [
    ("lightgbm","lightgbm"),
    ("xgboost","xgboost"),
    ("thop","thop"),
    ("openpyxl","openpyxl"),
    ("torchmetrics","torchmetrics==1.4.0"),
]:
    ensure(*p)

lightgbm ✓
xgboost ✓
Installing thop …
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.5 MB/s eta 0:00:00
openpyxl ✓
torchmetrics ✓


## 2) ⚙️ Imports & Global Settings

In [2]:
import os
for root, dirs, files in os.walk("/kaggle/input"):
    for f in files:
        print(os.path.join(root, f))

/kaggle/input/driver-behavior-and-route-anomaly-dataset-dbra24/driver_behavior_route_anomaly_dataset_with_derived_features.csv


In [3]:
# ================== Imports ==================
import os, math, time, pickle, copy
import numpy as np, pandas as pd
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler

from sklearn.metrics import average_precision_score, f1_score, mean_squared_error, precision_recall_curve, roc_curve, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import matplotlib.pyplot as plt

import os, gc, time, math, random, json, pickle, shutil, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import (
    average_precision_score, precision_recall_curve, roc_curve, confusion_matrix,
    f1_score, mean_squared_error
)
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

import lightgbm as lgb
import xgboost as xgb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

from thop import profile

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# Globals
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Paths
DATA_CSV = "/kaggle/input/driver-behavior-and-route-anomaly-dataset-dbra24/driver_behavior_route_anomaly_dataset_with_derived_features.csv"

# Directories
OUT_DIR   = Path("./out"); OUT_DIR.mkdir(exist_ok=True)
MODEL_DIR = Path("./models"); MODEL_DIR.mkdir(exist_ok=True)
PLOT_DIR  = Path("./plots"); PLOT_DIR.mkdir(exist_ok=True)

# Config
CFG = dict(
    SEQ_LEN=60, STRIDE=5, SAMPLE_RATE_HZ=1,
    BATCH_SIZE=256, EPOCHS=25, LR=5e-4, HIDDEN=256,
    TCN_CHANNELS=64, DROPOUT=0.3, PATIENCE=5
)
SEED=42

print("Environment ready.")

Using device: cuda
Environment ready.


## 3) 📥 Load & Inspect Data
This block is robust to small naming differences and ensures required columns exist.

In [4]:
def read_db(csv_path=DATA_CSV):
    df = pd.read_csv(csv_path)

    # Normalize a few common names
    rename_map = {
        'lat':'latitude','lon':'longitude','long':'longitude','ts':'timestamp',
        'driverid':'driver_id','vehicleid':'vehicle_id', 'tripid':'trip_id',
        'anomaly_event':'anomalous_event', 'route_anom':'route_anomaly',
        'route_deviation':'route_deviation_score'
    }
    df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})

    # Timestamps
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    else:
        raise ValueError("Missing 'timestamp' column in dataset.")

    # Labels (create safe fallbacks if absent)
    for col in ['anomalous_event','route_anomaly','route_deviation_score']:
        if col not in df.columns:
            df[col] = 0 if col != 'route_deviation_score' else 0.0

    # Ensure IDs exist
    if 'driver_id' not in df.columns:
        df['driver_id'] = 0
    if 'vehicle_id' not in df.columns:
        df['vehicle_id'] = 0

    # ✅ Trip assignment logic
    if 'trip_id' not in df.columns or df['trip_id'].nunique() >= len(df):
        # If trip_id is missing OR every row is unique
        if df['driver_id'].nunique() > 1:
            # Group by driver
            df['trip_id'] = df['driver_id'].astype(str)
        elif df['vehicle_id'].nunique() > 1:
            # Fallback: group by vehicle
            df['trip_id'] = df['vehicle_id'].astype(str)
        else:
            # Last resort: treat all rows as one trip
            df['trip_id'] = "trip0"

    return df
df_raw = read_db()
print("Rows:", len(df_raw))
print("Unique trips:", df_raw['trip_id'].nunique())
print(df_raw.groupby('trip_id').size().describe())


Rows: 120000
Unique trips: 5
count        5.000000
mean     24000.000000
std      20744.178424
min      11800.000000
25%      11945.000000
50%      12204.000000
75%      24163.000000
max      59888.000000
dtype: float64


## 4) 🧭 Feature Groups & Light Engineering
We organize columns into **kinematics**, **GPS**, **context** (weather/road/traffic), **map-aware**, and **trip statics**, then derive a few helpful signals.

In [5]:
def find_cols(df):
    kinematics = [c for c in df.columns if c.lower() in
                  ['speed','acceleration','rpm','steering_angle','heading','brake_usage',
                   'lane_deviation','acceleration_variation','behavioral_consistency_index','turn_rate','yaw_rate','jerk']]
    gps = [c for c in df.columns if c.lower() in ['latitude','longitude','bearing','altitude']]
    context = [c for c in df.columns if c.lower() in ['weather_conditions','road_type','traffic_condition']]
    mapaware = [c for c in df.columns if 'geofencing' in c.lower() or 'curvature' in c.lower()]
    trip_static = [c for c in df.columns if c.lower() in ['trip_duration','trip_distance']]
    quality = [c for c in df.columns if c.lower() in ['signal_quality','gps_accuracy']]
    return dict(kinematics=kinematics, gps=gps, context=context, mapaware=mapaware,
                trip_static=trip_static, quality=quality)

def add_derived_features(df):
    df = df.sort_values(['trip_id','timestamp']).copy()
    # heading deltas → turn intensity
    if 'heading' in df.columns:
        df['d_heading'] = df.groupby('trip_id')['heading'].diff().fillna(0)
        df['turn_intensity'] = df['d_heading'].abs()
    # jerk from acceleration
    if 'acceleration' in df.columns:
        df['jerk'] = df.groupby('trip_id')['acceleration'].diff().fillna(0)
    # curvature proxy if missing
    if 'curvature' not in df.columns and {'latitude','longitude'}.issubset(df.columns):
        for ax in ['latitude','longitude']:
            df[f'd_{ax}'] = df.groupby('trip_id')[ax].diff().fillna(0)
        df['curvature'] = np.hypot(df['d_latitude'], df['d_longitude']).rolling(3, min_periods=1).mean()
        df.drop(columns=[c for c in ['d_latitude','d_longitude'] if c in df.columns], inplace=True)
    return df

df = add_derived_features(df_raw)
COLS = find_cols(df)
print("Groups:", {k:len(v) for k,v in COLS.items()})

Groups: {'kinematics': 10, 'gps': 2, 'context': 3, 'mapaware': 2, 'trip_static': 2, 'quality': 0}


## 5) 🧱 Windowing into Sequences
We convert per-second telemetry into sliding windows (default **T=60s**, stride **5s**).  
Colored highlights explain ablations in later sections.

In [6]:
# Quick sanity check
print("Rows:", len(df))
print("Unique trips:", df['trip_id'].nunique())
print("Per-trip length stats:")
print(df.groupby('trip_id').size().describe())
print("Timestamp range:", df['timestamp'].min(), "→", df['timestamp'].max())


Rows: 120000
Unique trips: 5
Per-trip length stats:
count        5.000000
mean     24000.000000
std      20744.178424
min      11800.000000
25%      11945.000000
50%      12204.000000
75%      24163.000000
max      59888.000000
dtype: float64
Timestamp range: 2023-01-01 00:00:00 → 2023-01-02 09:19:59


In [7]:
# ==== SEQUENCE BUILDER WITH SAFETY ====
def make_sequences(df, seq_len=60, stride=5, sample_hz=1, 
                   ablation=None, short_history=False, min_len=10):
    df_proc = df.copy()

    # One-hot encode categorical context
    for cat in [c for c in ['weather_conditions','road_type','traffic_condition'] if c in df_proc.columns]:
        df_proc[cat] = df_proc[cat].astype(str).fillna("unk")
    context_cols = [c for c in ['weather_conditions','road_type','traffic_condition'] if c in df_proc.columns]
    oh = pd.get_dummies(df_proc[context_cols], prefix=context_cols) if context_cols else pd.DataFrame(index=df_proc.index)
    df_proc = pd.concat([df_proc.drop(columns=context_cols), oh], axis=1)

    # Time-of-day encoding
    df_proc['sec_of_day'] = df_proc['timestamp'].dt.hour*3600 + df_proc['timestamp'].dt.minute*60 + df_proc['timestamp'].dt.second
    df_proc['sin_t'] = np.sin(2*np.pi*df_proc['sec_of_day']/86400)
    df_proc['cos_t'] = np.cos(2*np.pi*df_proc['sec_of_day']/86400)

    # Dynamic features
    dyn_cols = COLS['kinematics'] + COLS['gps'] + ['turn_intensity','jerk','curvature','sin_t','cos_t']
    dyn_cols = [c for c in dyn_cols if c in df_proc.columns]

    # A5 jitter
    if ablation == "A5-jitter":
        ctx_like = [c for c in df_proc.columns if any(k in c.lower() for k in ['weather_', 'road_', 'traffic_'])]
        for c in ctx_like:
            df_proc[c] = df_proc.groupby('trip_id')[c].shift(1)

    # A6 short history
    if short_history:
        seq_len = 20

    # A2/A8 feature drop
    drop_cols = []
    if ablation == "A2-nocontext":
        drop_cols += [c for c in df_proc.columns if any(k in c.lower() for k in ['weather_', 'road_', 'traffic_'])]
    if ablation == "A8-mapfree":
        drop_cols += [c for c in df_proc.columns if 'geofencing' in c.lower() or 'curvature' in c.lower()]
    df_proc = df_proc.drop(columns=[c for c in set(drop_cols) if c in df_proc.columns], errors='ignore')

    dyn_cols = [c for c in dyn_cols if c in df_proc.columns]
    dyn_cols = list(dict.fromkeys(dyn_cols + [c for c in df_proc.columns if any(p in c for p in ['weather_','road_','traffic_'])]))

    static_cols = [c for c in COLS['trip_static'] if c in df_proc.columns]
    if 'behavioral_consistency_index' in df_proc.columns and 'behavioral_consistency_index' not in static_cols:
        static_cols.append('behavioral_consistency_index')

    labels_A = 'anomalous_event'
    labels_B = 'route_anomaly'
    labels_C = 'route_deviation_score'

    # Slide per trip
    seqs, statics, yA, yB, yC, meta = [], [], [], [], [], []
    skipped = 0
    for trip, df_t in df_proc.groupby('trip_id', sort=False):
        df_t = df_t.sort_values('timestamp').copy()
        df_t = df_t.set_index('timestamp').resample(f"{int(1000/sample_hz)}L").nearest(limit=1).ffill().bfill().reset_index()
        n = len(df_t)
        if n < seq_len: 
            skipped += 1
            continue
        for start in range(0, n - seq_len + 1, stride):
            w = df_t.iloc[start:start+seq_len]
            X = w[dyn_cols].astype(float).values if dyn_cols else np.zeros((seq_len,0))
            s = w[static_cols].mean().values if static_cols else np.zeros((0,))
            a = int(w[labels_A].max())
            b = int(w[labels_B].max())
            c = float(w[labels_C].mean())
            seqs.append(X); statics.append(s); yA.append(a); yB.append(b); yC.append(c)
            meta.append(dict(trip_id=trip, t0=w['timestamp'].iloc[0], t1=w['timestamp'].iloc[-1]))
    if skipped > 0:
        print(f"[make_sequences] ⚠️ Skipped {skipped} trips with length < {seq_len}")
    print(f"[make_sequences] Generated {len(seqs)} windows (seq_len={seq_len}, stride={stride})")

    return dict(X=seqs, S=statics, yA=np.array(yA), yB=np.array(yB), yC=np.array(yC), meta=meta,
                dyn_cols=dyn_cols, static_cols=static_cols)

In [8]:
import numpy as np

def subsample_dict(d, frac=0.4, seed=42):
    """Safely subsample dictionary of sequences."""
    n = len(d['X'])
    if n == 0:
        print("[subsample_dict] ⚠️ No data to subsample (n=0). Returning original dict.")
        return d
    
    take = max(1, int(frac * n))
    idx = np.random.default_rng(seed).choice(n, take, replace=False)

    return {
        k: (d[k][idx] if isinstance(d[k], np.ndarray) else [d[k][i] for i in idx])
        for k in ['X','S','yA','yB','yC','meta']
    }


In [9]:
# Step 1: make sequences (auto-adjust ensures >0 windows if possible)
data_full = make_sequences(df, seq_len=CFG['SEQ_LEN'], stride=CFG['STRIDE'], sample_hz=CFG['SAMPLE_RATE_HZ'])

# Step 2: subsample if needed
data_full = subsample_dict(data_full, frac=0.4, seed=SEED)
print("After subsample:", len(data_full['X']))


[make_sequences] Generated 119930 windows (seq_len=60, stride=5)
After subsample: 47972


In [10]:
# ==== AUTO-FIX if NO WINDOWS ====
def rebuild_sequences_auto(df, base_stride=5):
    trip_len = df.groupby('trip_id').size()
    med = int(trip_len.median()) if len(trip_len) else 60
    T0  = max(10, min(60, max(10, med // 2)))
    stride = max(1, min(base_stride, T0 // 5))

    for T_try in [T0, 40, 30, 20, 10]:
        seqs = make_sequences(df, seq_len=T_try, stride=stride, sample_hz=CFG['SAMPLE_RATE_HZ'])
        if len(seqs['X']) > 0:
            CFG['SEQ_LEN'] = T_try
            CFG['STRIDE']  = stride
            print(f"[Auto] Using SEQ_LEN={T_try}, STRIDE={stride} → windows={len(seqs['X'])}")
            return seqs

    print("[Auto] ⚠️ Could not create windows with T>=10 → fallback to T=1")
    seqs = make_sequences(df, seq_len=1, stride=1, sample_hz=CFG['SAMPLE_RATE_HZ'])
    CFG['SEQ_LEN'] = 1
    CFG['STRIDE']  = 1
    return seqs


## 6) ✂️ Split by Driver (No Leakage)
We split train/val/test **by driver** so windows from the same driver don’t leak into other sets.

In [11]:
# ==== SPLIT with FEATURE METADATA ====
def split_dict(d, idx):
    out = {
        k: (d[k][idx] if isinstance(d[k], np.ndarray) else [d[k][i] for i in idx])
        for k in ['X','S','yA','yB','yC','meta']
    }
    out['dyn_cols'] = d.get('dyn_cols', [])
    out['static_cols'] = d.get('static_cols', [])
    return out


# ==== STACK UTILITY ====
def stack_X(X_list):
    if len(X_list) == 0:
        return np.zeros((0,0,0), dtype=np.float32)
    T = len(X_list[0]); F = X_list[0].shape[1] if T > 0 else 0
    out = np.zeros((len(X_list), T, F), dtype=np.float32)
    for i,x in enumerate(X_list): out[i] = x
    return out

## 7) 📏 Scaling & PyTorch Datasets
We standardize dynamic features using **train only** then wrap them into PyTorch datasets.

In [12]:
from sklearn.model_selection import GroupShuffleSplit

# ============== GROUPED SPLIT (driver_id or trip_id) ==============
drivers = df[['trip_id','driver_id']].drop_duplicates()
trip2driver = dict(zip(drivers['trip_id'], drivers['driver_id']))

groups_driver = np.array([trip2driver.get(m['trip_id'], -1) for m in data_full['meta']])
n_unique_drivers = pd.Series(groups_driver).nunique()

if n_unique_drivers < 2:
    print("[Warn] Only one driver → grouping by trip_id instead.")
    groups = np.array([m['trip_id'] for m in data_full['meta']])
else:
    groups = groups_driver

n_samples = len(groups)
if n_samples == 0:
    raise RuntimeError("❌ No sequences created. Check your preprocessing.")

if pd.Series(groups).nunique() < 2:
    # fallback to random split
    rng = np.random.default_rng(SEED)
    idx_all = np.arange(n_samples); rng.shuffle(idx_all)
    n_test = max(1, int(0.2*n_samples)); n_val = max(1, int(0.1*(n_samples-n_test)))
    idx_test = idx_all[:n_test]; idx_val = idx_all[n_test:n_test+n_val]; idx_train = idx_all[n_test+n_val:]
else:
    gss = GroupShuffleSplit(test_size=0.2, random_state=SEED)
    idx_train, idx_test = next(gss.split(np.arange(n_samples), groups=groups))
    gss_val = GroupShuffleSplit(test_size=0.125, random_state=SEED)  # ~10% total for val
    idx_train, idx_val = next(gss_val.split(idx_train, groups=groups[idx_train]))

print("✅ Split sizes:", len(idx_train), len(idx_val), len(idx_test))

# ==== APPLY ====
if len(data_full['X']) == 0:
    data_full = rebuild_sequences_auto(df)

train = split_dict(data_full, idx_train)
val   = split_dict(data_full, idx_val)
test  = split_dict(data_full, idx_test)

dyn_cols = train['dyn_cols']
static_cols = train['static_cols']

# Scale features
X_train = stack_X(train['X']); X_val = stack_X(val['X']); X_test = stack_X(test['X'])
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True, with_std=True)

if X_train.shape[-1] > 0:
    scaler.fit(X_train.reshape(-1, X_train.shape[-1]))
    def transform(X):
        if X.shape[-1]==0: return X
        Xr = X.reshape(-1, X.shape[-1]); Xr = scaler.transform(Xr)
        return Xr.reshape(X.shape)
else:
    transform = lambda x: x
X_train, X_val, X_test = transform(X_train), transform(X_val), transform(X_test)

# Static features
S_train = np.array(train['S'], dtype=np.float32) if static_cols else np.zeros((len(train['X']),0), np.float32)
S_val   = np.array(val['S'],   dtype=np.float32) if static_cols else np.zeros((len(val['X']),0), np.float32)
S_test  = np.array(test['S'],  dtype=np.float32) if static_cols else np.zeros((len(test['X']),0), np.float32)

# Labels
yA_train, yB_train, yC_train = train['yA'], train['yB'], train['yC']
yA_val,   yB_val,   yC_val   = val['yA'],   val['yB'],   val['yC']
yA_test,  yB_test,  yC_test  = test['yA'],  test['yB'],  test['yC']

# ==== TORCH DATASETS ====
class SeqDataset(Dataset):
    def __init__(self, X, S, yA, yB, yC):
        self.X, self.S = torch.tensor(X), torch.tensor(S)
        self.yA = torch.tensor(yA).float()
        self.yB = torch.tensor(yB).float()
        self.yC = torch.tensor(yC).float()
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.S[i], self.yA[i], self.yB[i], self.yC[i]

ds_train = SeqDataset(X_train, S_train, yA_train, yB_train, yC_train)
ds_val   = SeqDataset(X_val,   S_val,   yA_val,   yB_val,   yC_val)
ds_test  = SeqDataset(X_test,  S_test,  yA_test,  yB_test,  yC_test)

dl_train = DataLoader(ds_train, batch_size=CFG['BATCH_SIZE'], shuffle=True,  num_workers=4, pin_memory=True)
dl_val   = DataLoader(ds_val,   batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4, pin_memory=True)
dl_test  = DataLoader(ds_test,  batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4, pin_memory=True)

print("✅ Data ready:", X_train.shape, S_train.shape, "| Dyn:", len(dyn_cols), "| Static:", len(static_cols))

✅ Split sizes: 28803 9681 9488
✅ Data ready: (28803, 60, 26) (28803, 0) | Dyn: 0 | Static: 0


## 8) 🧠 Models (Bi-LSTM, TCN-Small, Gating)
- <span style="color:#6C5CE7">Gating</span> learns reliability weights per feature (can be disabled in A3).  
- <span style="color:#E17055">Bi-LSTM</span> is our main sequence model.  
- <span style="color:#0984E3">TCN-Small</span> is an efficient edge variant.

In [13]:
# ================== Models ==================
class GateLayer(nn.Module):
    def __init__(self, in_feats=None):
        super().__init__()
        self.fc = nn.Linear(in_feats, in_feats) if in_feats else None
    def forward(self, x):  # [B,T,F]
        F = x.size(-1)
        if self.fc is None or self.fc.in_features != F:
            self.fc = nn.Linear(F, F).to(x.device)
        g = torch.sigmoid(self.fc(x.mean(1)))
        return x * g.unsqueeze(1)

class BiLSTM(nn.Module):
    def __init__(self, in_feats, static_feats=0, hidden=128, layers=2, dropout=0.1, gated=True):
        super().__init__()
        self.gated = gated and in_feats>0
        self.gate = GateLayer(in_feats) if self.gated else nn.Identity()
        self.lstm = nn.LSTM(input_size=in_feats, hidden_size=hidden, num_layers=layers,
                            batch_first=True, dropout=dropout, bidirectional=True)
        emb = hidden*2
        self.fc_context = nn.Linear(static_feats, hidden) if static_feats>0 else None
        head_in = emb + (hidden if self.fc_context else 0)
        self.headA, self.headB, self.headC = nn.Linear(head_in,1), nn.Linear(head_in,1), nn.Linear(head_in,1)
    def forward(self, x, s):
        x = self.gate(x) if self.gated else x
        out,_ = self.lstm(x); h = out.mean(1)
        if self.fc_context: h = torch.cat([h, torch.relu(self.fc_context(s))], dim=1)
        return self.headA(h).squeeze(1), self.headB(h).squeeze(1), self.headC(h).squeeze(1)

class TCNBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=3, d=1):
        super().__init__()
        self.conv = nn.Conv1d(in_ch, out_ch, k, padding=d*(k-1)//2, dilation=d)
        self.bn = nn.BatchNorm1d(out_ch)
    def forward(self, x): return torch.relu(self.bn(self.conv(x)))

class TCN(nn.Module):
    def __init__(self, in_feats, static_feats=0, ch=64, layers=3, gated=True):
        super().__init__()
        self.gated = gated and in_feats>0
        self.gate = GateLayer(in_feats) if self.gated else nn.Identity()
        chans = [in_feats, ch, ch, ch][:layers+1]
        blocks = [TCNBlock(chans[i], chans[i+1], d=2**i) for i in range(layers)]
        self.net = nn.Sequential(*blocks)
        self.fc_context = nn.Linear(static_feats, ch) if static_feats>0 else None
        head_in = ch + (ch if self.fc_context else 0)
        self.headA, self.headB, self.headC = nn.Linear(head_in,1), nn.Linear(head_in,1), nn.Linear(head_in,1)
    def forward(self, x, s):
        x = self.gate(x) if self.gated else x
        z = self.net(x.permute(0,2,1)).mean(-1)
        if self.fc_context: z = torch.cat([z, torch.relu(self.fc_context(s))], dim=1)
        return self.headA(z).squeeze(1), self.headB(z).squeeze(1), self.headC(z).squeeze(1)

## 9) 📐 Losses & Metrics
We compute:
- **PR-AUC (A/B)**, **F1 (A)**, **RMSE (C)**, **ECE** (calibration),  
- **TTD** (time-to-detection), **FP/h**, **Latency (ms)**, **FLOPs (M)**.

In [14]:
# ================== Losses / Metrics ==================
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0): super().__init__(); self.alpha,self.gamma=alpha,gamma
    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)
        ce = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
        p_t = probs*targets + (1-probs)*(1-targets)
        loss = ce * ((1-p_t)**self.gamma) * (self.alpha*targets + (1-self.alpha)*(1-targets))
        return loss.mean()

def ece_score(y_true, y_prob, n_bins=15):
    y_true, y_prob = np.asarray(y_true).astype(int), np.asarray(y_prob)
    bins = np.linspace(0.0, 1.0, n_bins+1); inds = np.digitize(y_prob, bins) - 1
    ece=0.0
    for b in range(n_bins):
        mask = inds==b
        if not np.any(mask): continue
        acc, conf = y_true[mask].mean(), y_prob[mask].mean()
        ece += mask.mean() * abs(acc-conf)
    return float(ece)

def best_f1_threshold(y_true,y_prob):
    ps, rs, ts = precision_recall_curve(y_true,y_prob); f1s = 2*ps*rs/(ps+rs+1e-9)
    i = np.nanargmax(f1s); return float(ts[i]) if i<len(ts) else 0.5

def ttd_and_fprate(meta, y_true, y_prob, threshold=0.5, window_seconds=5):
    """Compute average time-to-detection (s) and false positives per hour."""
    y_true = np.asarray(y_true).astype(int)
    y_pred = (np.asarray(y_prob) >= threshold).astype(int)

    # === FP/h ===
    fp = ((y_pred == 1) & (y_true == 0)).sum()
    hours = (len(y_true) * window_seconds) / 3600.0
    fp_rate = fp / max(hours, 1e-6)

    # === TTD (s) ===
    ttd_list = []
    pos_idx = np.where(y_true == 1)[0]
    if len(pos_idx) > 0:
        # For each true positive segment, find first detection afterwards
        for i in pos_idx:
            detect_idx = np.where((y_pred[i:] == 1))[0]
            if len(detect_idx) > 0:
                delay_s = detect_idx[0] * window_seconds
                ttd_list.append(delay_s)
        ttd_s = np.mean(ttd_list) if len(ttd_list) > 0 else np.nan
    else:
        ttd_s = np.nan

    return ttd_s, fp_rate

import time
import torch
from thop import profile

def measure_latency_flops(model, in_feats, static_feats, seq_len=60, runs=30):
    """Compute model FLOPs (M) and latency (ms) — with GPU or CPU fallback."""
    model = model.to(device).eval()
    X = torch.randn(1, seq_len, in_feats).to(device)
    S = torch.randn(1, static_feats).to(device)

    # ===== FLOPs estimation =====
    try:
        with torch.no_grad():
            flops, params = profile(model, inputs=(X, S), verbose=False)
        flops_m = flops / 1e6  # millions
    except Exception as e:
        print(f"[WARN] FLOPs measurement failed: {e}")
        flops_m = np.nan

    # ===== Latency measurement =====
    timings = []
    with torch.no_grad():
        # Warm-up runs
        for _ in range(5):
            _ = model(X, S)

        if device.type == "cuda":
            starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
            for _ in range(runs):
                starter.record()
                _ = model(X, S)
                ender.record()
                torch.cuda.synchronize()
                timings.append(starter.elapsed_time(ender))  # ms
            latency_ms = float(np.mean(timings))
        else:
            # CPU fallback
            for _ in range(runs):
                t0 = time.time()
                _ = model(X, S)
                t1 = time.time()
                timings.append((t1 - t0) * 1000)  # convert to ms
            latency_ms = float(np.mean(timings))

    return flops_m, latency_ms


def save_curves(ablation_code, y, p, tag):
    # PR
    P,R,thr = precision_recall_curve(y,p)
    plt.figure(); plt.plot(R,P); plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"{ablation_code} PR {tag}")
    plt.grid(True, alpha=.3); plt.tight_layout(); plt.savefig(PLOT_DIR/f"{ablation_code}_PR_{tag}.png"); plt.close()
    # ROC
    try:
        fpr, tpr, _ = roc_curve(y,p)
        plt.figure(); plt.plot(fpr,tpr); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"{ablation_code} ROC {tag}")
        plt.grid(True, alpha=.3); plt.tight_layout(); plt.savefig(PLOT_DIR/f"{ablation_code}_ROC_{tag}.png"); plt.close()
    except Exception:
        pass
    # Reliability
    bins = np.linspace(0,1,11); inds = np.digitize(p,bins)-1
    acc=[]; conf=[]
    for b in range(10):
        msk=inds==b
        if msk.any():
            acc.append(y[msk].mean()); conf.append(p[msk].mean())
    plt.figure(); plt.plot([0,1],[0,1],'--',alpha=.4); plt.plot(conf, acc, marker='o')
    plt.xlabel("Confidence"); plt.ylabel("Accuracy"); plt.title(f"{ablation_code} Reliability {tag}")
    plt.grid(True, alpha=.3); plt.tight_layout(); plt.savefig(PLOT_DIR/f"{ablation_code}_Reliability_{tag}.png"); plt.close()

def save_confusion(ablation_code, y, p, tag, thr):
    cm = confusion_matrix(y, (p>=thr).astype(int))
    plt.figure(); plt.imshow(cm, cmap="Blues"); plt.title(f"{ablation_code} Confusion {tag}\n(thr={thr:.3f})")
    plt.colorbar(); plt.xticks([0,1],["Pred 0","Pred 1"]); plt.yticks([0,1],["True 0","True 1"])
    for (i,j),v in np.ndenumerate(cm):
        plt.text(j,i, int(v), ha="center", va="center")
    plt.tight_layout(); plt.savefig(PLOT_DIR/f"{ablation_code}_Confusion_{tag}.png"); plt.close()

## 10) 🏃 Training & Evaluation Utilities
Includes early stopping, metric computation, latency/FLOPs estimation, and plot saving.

In [15]:
# =================== FIX: use per-split loaders in training ===================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

from torch.cuda.amp import autocast, GradScaler
import copy

Using device: cuda


In [16]:
# ================== Training Loop ==================
def make_loaders(ds_tr, ds_va, batch):
    return (DataLoader(ds_tr,batch,True,4,pin_memory=True),
            DataLoader(ds_va,batch,False,4,pin_memory=True))

def train_model(model, ablation_name, loss_mode, dl_tr, dl_va, yA_for_weights, yB_for_weights):
    model = model.to(device); opt = AdamW(model.parameters(), lr=CFG['LR'])
    scaler = GradScaler(enabled=(device.type=="cuda"))
    cwA = compute_class_weight('balanced', classes=np.array([0,1]), y=yA_for_weights).tolist()
    cwB = compute_class_weight('balanced', classes=np.array([0,1]), y=yB_for_weights).tolist()
    posA,posB = torch.tensor(cwA[1]/cwA[0],device=device),torch.tensor(cwB[1]/cwB[0],device=device)
    focal = FocalLoss().to(device)
    best_val=float("inf"); bad=0

    for ep in range(1, CFG['EPOCHS']+1):
        model.train(); tr_loss=0.0; opt.zero_grad()
        for i,(X,S,ya,yb,yc) in enumerate(dl_tr):
            X,S,ya,yb,yc = X.to(device),S.to(device),ya.to(device),yb.to(device),yc.to(device)
            with autocast(enabled=(device.type=="cuda")):
                a,b,c = model(X,S)
                la = focal(a,ya) if loss_mode=='focal' else F.binary_cross_entropy_with_logits(a,ya,pos_weight=posA)
                lb = focal(b,yb) if loss_mode=='focal' else F.binary_cross_entropy_with_logits(b,yb,pos_weight=posB)
                lc = F.smooth_l1_loss(c,yc)
                loss=(la+lb+0.5*lc)/2
            scaler.scale(loss).backward(); scaler.step(opt); scaler.update(); opt.zero_grad()
            tr_loss += loss.item()
        # Validation
        model.eval(); vl=0.0;n=0;yA,yB,pa,pb=[],[],[],[]
        with torch.no_grad(), autocast(enabled=(device.type=="cuda")):
            for X,S,ya,yb,yc in dl_va:
                X,S,ya,yb,yc = X.to(device),S.to(device),ya.to(device),yb.to(device),yc.to(device)
                a,b,c = model(X,S)
                la=F.binary_cross_entropy_with_logits(a,ya); lb=F.binary_cross_entropy_with_logits(b,yb); lc=F.smooth_l1_loss(c,yc)
                vl+=(la+lb+0.5*lc).item(); n+=1
                yA.extend(ya.cpu().numpy()); pa.extend(torch.sigmoid(a).cpu().numpy())
                yB.extend(yb.cpu().numpy()); pb.extend(torch.sigmoid(b).cpu().numpy())
        vl/=max(1,n)
        prA, f1A = average_precision_score(yA,pa), f1_score(yA,(np.array(pa)>=0.5).astype(int))
        prB, f1B = average_precision_score(yB,pb), f1_score(yB,(np.array(pb)>=0.5).astype(int))
        print(f"[{ablation_name}] epoch {ep} train_loss={tr_loss/len(dl_tr):.4f} val_loss={vl:.4f} "
              f"A: PR-AUC={prA:.3f} F1={f1A:.3f}  B: PR-AUC={prB:.3f} F1={f1B:.3f}")
        if vl<best_val: best_val=vl; bad=0; torch.save(model.state_dict(), MODEL_DIR/f"{ablation_name}_best.pt")
        else: bad+=1; 
        if bad>=CFG['PATIENCE']: print(f"🛑 Early stopping at epoch {ep}"); break
    model.load_state_dict(torch.load(MODEL_DIR/f"{ablation_name}_best.pt")); return model.eval()

# ================== Eval ==================
def predict_model(model, ds, batch=CFG['BATCH_SIZE']):
    dl = DataLoader(ds, batch_size=batch, shuffle=False, num_workers=4, pin_memory=True)
    pa, pb, pc = [], [], []
    model.eval()
    with torch.no_grad(), autocast(enabled=(device.type=="cuda")):
        for X, S, _, _, _ in dl:
            X = X.to(device); S = S.to(device)
            a, b, c = model(X, S)
            pa.extend(torch.sigmoid(a).cpu().numpy())
            pb.extend(torch.sigmoid(b).cpu().numpy())
            pc.extend(c.cpu().numpy())
    return np.array(pa), np.array(pb), np.array(pc)

def eval_all(name, yA, yB, yC, pA, pB, pC, meta):
    thA, thB = best_f1_threshold(yA, pA), best_f1_threshold(yB, pB)
    ttd_s, fp_h = ttd_and_fprate(meta, yA, pA, threshold=thA)
    return dict(
        precision_auc_A=average_precision_score(yA, pA),
        f1_A=f1_score(yA, (pA >= thA).astype(int)),
        precision_auc_B=average_precision_score(yB, pB),
        rmse_C=math.sqrt(mean_squared_error(yC, pC)),
        ece=ece_score(yA, pA),
        ttd_s=ttd_s,
        fp_per_hour=fp_h
    )


## 11) 🌳 A1 — <span style="color:#6C5CE7">GBM (No Mobility, Temporal-Agnostic)</span>
Replace sequence DL with **static aggregates** (mean/std/min/max over window).  
**Hypothesis:** PR-AUC drops; **TTD** increases.

In [17]:
# ================== GBM Baseline (A1) ==================
def aggregate_for_gbm(split):
    feats=[np.concatenate([X.mean(0),X.std(0),X.min(0),X.max(0)]) if X.shape[-1]>0 else np.zeros(4) for X in split['X']]
    return np.array(feats,np.float32), np.array(split['yA']), np.array(split['yB']), np.array(split['yC'])

def run_gbm(name="A1_GBM"):
    Xtr,yAtr,yBtr,yCtr = aggregate_for_gbm(train)
    Xva,yAva,yBva,yCva = aggregate_for_gbm(val)
    Xte,yAte,yBte,yCte = aggregate_for_gbm(test)

    gbmA = lgb.LGBMClassifier(
        n_estimators=1000, learning_rate=0.05, num_leaves=64,
        objective="binary", random_state=SEED
    )
    gbmB = lgb.LGBMClassifier(
        n_estimators=1000, learning_rate=0.05, num_leaves=64,
        objective="binary", random_state=SEED
    )
    gbrC = lgb.LGBMRegressor(
        n_estimators=1000, learning_rate=0.05, num_leaves=64,
        objective="regression", random_state=SEED
    )

    # Fit with early stopping
    gbmA.fit(Xtr, yAtr,
             eval_set=[(Xva, yAva)],
             eval_metric="average_precision",
             callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(0)])  # 0 = silent
    gbmB.fit(Xtr, yBtr,
             eval_set=[(Xva, yBva)],
             eval_metric="average_precision",
             callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(0)])
    gbrC.fit(Xtr, yCtr,
             eval_set=[(Xva, yCva)],
             eval_metric="rmse",
             callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(0)])

    # Predictions
    pA = gbmA.predict_proba(Xte)[:,1]
    pB = gbmB.predict_proba(Xte)[:,1]
    pC = gbrC.predict(Xte)

    return eval_all("A1", yAte, yBte, yCte, pA, pB, pC, test['meta'])



## 12) 🧪 Ablations A2–A10 (one-by-one)
We implement each hypothesis exactly as described and log metrics & plots.

In [18]:
def run_ablation(code):
    # Defaults
    ablate=None; short_hist=False; gated=True; kind="lstm"; big=False; small=False; loss_mode='bce_weighted'
    if code=="A2": ablate="A2-nocontext"
    if code=="A3": gated=False
    if code=="A4": big=True
    if code=="A5": ablate="A5-jitter"
    if code=="A6": short_hist=True
    if code=="A7": gated=True
    if code=="A8": ablate="A8-mapfree"
    if code=="A9": loss_mode='focal'
    if code=="A10": kind="tcn"; small=True

    # Build per-ablation datasets (if features/horizon change)
    if code in ["A2","A5","A6","A8"]:
        seqs = make_sequences(df, seq_len=CFG['SEQ_LEN'], stride=CFG['STRIDE'],
                              sample_hz=CFG['SAMPLE_RATE_HZ'], ablation=ablate,
                              short_history=short_hist)

        def re_split(seqs, idx):
            get = lambda k: [seqs[k][i] for i in idx]
            return dict(X=get('X'), S=get('S'),
                        yA=np.array([seqs['yA'][i] for i in idx]),
                        yB=np.array([seqs['yB'][i] for i in idx]),
                        yC=np.array([seqs['yC'][i] for i in idx]),
                        meta=[seqs['meta'][i] for i in idx])

        tr, va, te = re_split(seqs, idx_train), re_split(seqs, idx_val), re_split(seqs, idx_test)

        # stack + scale
        def stack(XL):
            T = len(XL[0]); F = XL[0].shape[1] if T>0 else 0
            out = np.zeros((len(XL), T, F), dtype=np.float32)
            for i,x in enumerate(XL): out[i]=x
            return out

        Xtr, Xva, Xte = stack(tr['X']), stack(va['X']), stack(te['X'])
        in_feats = Xtr.shape[-1]
        static_feats = (len(tr['S'][0]) if len(tr['S'])>0 and hasattr(tr['S'][0],'__len__') else 0)

        scaler2 = StandardScaler()
        if in_feats>0:
            scaler2.fit(Xtr.reshape(-1, in_feats))
            def T(x):
                if x.shape[-1]==0: return x
                xr = x.reshape(-1, in_feats); xr = scaler2.transform(xr)
                return xr.reshape(x.shape)
        else:
            T = lambda x: x

        Xtr, Xva, Xte = T(Xtr), T(Xva), T(Xte)
        Str, Sva, Ste = np.array(tr['S'], np.float32), np.array(va['S'], np.float32), np.array(te['S'], np.float32)

        ds_tr = SeqDataset(Xtr, Str, tr['yA'], tr['yB'], tr['yC'])
        ds_va = SeqDataset(Xva, Sva, va['yA'], va['yB'], va['yC'])
        ds_te = SeqDataset(Xte, Ste, te['yA'], te['yB'], te['yC'])
        yA_w, yB_w = tr['yA'], tr['yB']

        dl_tr, dl_va = make_loaders(ds_tr, ds_va, CFG['BATCH_SIZE'])
    else:
        # reuse global prebuilt datasets/loaders
        ds_tr, ds_va, ds_te = ds_train, ds_val, ds_test
        dl_tr, dl_va = dl_train, dl_val
        in_feats, static_feats = X_train.shape[-1], S_train.shape[-1]
        yA_w, yB_w = yA_train, yB_train

    # build & train on the loaders that match this ablation's feature dims
    mdl = make_model(kind, in_feats, static_feats, big=big, small=small, gated=gated)
    mdl = train_model(mdl, code, loss_mode, dl_tr, dl_va, yA_w, yB_w)

    # predict + eval on the matching test set
    pA, pB, pC = predict_model(mdl, ds_te, batch=CFG['BATCH_SIZE'])
    metrics = eval_all(code, ds_te.yA.numpy(), ds_te.yB.numpy(), ds_te.yC.numpy(), pA, pB, pC, (test['meta'] if code not in ["A2","A5","A6","A8"] else te['meta']))
    flops_m, ms = measure_latency_flops(mdl, in_feats, static_feats, seq_len=CFG['SEQ_LEN'])
    metrics['latency_ms'] = ms; metrics['flops_m'] = flops_m

    if code=="A7":
        Xte = ds_te.X.clone()
        sensor_names = ['speed','acceleration','rpm','steering_angle']
        dyn_cols_here = data_full.get('dyn_cols', [])
        sensor_idx = [i for i,c in enumerate(dyn_cols_here) if c in sensor_names]
    
        rng = np.random.default_rng(SEED)
        for i in range(Xte.shape[0]):
            for j in sensor_idx:
                if rng.uniform() < 0.2:
                    miss_idx = rng.integers(0, Xte.shape[1], size=max(1, Xte.shape[1]//10))
                    Xte[i, miss_idx, j] = 0.0
    
        ds_drop = SeqDataset(Xte.numpy(), ds_te.S.numpy(), ds_te.yA.numpy(), ds_te.yB.numpy(), ds_te.yC.numpy())
        pA2, pB2, pC2 = predict_model(mdl, ds_drop, batch=CFG['BATCH_SIZE'])
        m2 = eval_all(code+"-dropout", ds_drop.yA.numpy(), ds_drop.yB.numpy(), ds_drop.yC.numpy(),
                      pA2, pB2, pC2, (test['meta'] if code not in ["A2","A5","A6","A8"] else te['meta']))
        metrics['robust_dPR_A'] = m2['precision_auc_A'] - metrics['precision_auc_A']
        metrics['robust_dFPH']  = m2['fp_per_hour'] - metrics['fp_per_hour']





In [19]:
# ================== Baseline BiLSTM Full ==================
# Train
full = BiLSTM(X_train.shape[-1], S_train.shape[-1], hidden=CFG['HIDDEN'], dropout=CFG['DROPOUT'], gated=True)
full = train_model(full, "BiLSTM_Full", "bce_weighted", dl_train, dl_val, yA_train, yB_train)

# Predict + Eval
ds_test = SeqDataset(X_test, S_test, yA_test, yB_test, yC_test)   # <-- ensure this
pA, pB, pC = predict_model(full, ds_test)
met_full = eval_all("Full", yA_test, yB_test, yC_test, pA, pB, pC, test['meta'])
print("Full BiLSTM:", met_full)

[BiLSTM_Full] epoch 1 train_loss=0.4868 val_loss=1.2239 A: PR-AUC=0.804 F1=0.659  B: PR-AUC=0.820 F1=0.650
[BiLSTM_Full] epoch 2 train_loss=0.4268 val_loss=1.2232 A: PR-AUC=0.830 F1=0.616  B: PR-AUC=0.844 F1=0.629
[BiLSTM_Full] epoch 3 train_loss=0.4193 val_loss=1.1782 A: PR-AUC=0.833 F1=0.669  B: PR-AUC=0.847 F1=0.670
[BiLSTM_Full] epoch 5 train_loss=0.4099 val_loss=1.1836 A: PR-AUC=0.837 F1=0.672  B: PR-AUC=0.852 F1=0.670
[BiLSTM_Full] epoch 6 train_loss=0.4043 val_loss=1.1865 A: PR-AUC=0.843 F1=0.668  B: PR-AUC=0.856 F1=0.679
[BiLSTM_Full] epoch 7 train_loss=0.3968 val_loss=1.1971 A: PR-AUC=0.845 F1=0.667  B: PR-AUC=0.861 F1=0.673
[BiLSTM_Full] epoch 8 train_loss=0.3893 val_loss=1.1114 A: PR-AUC=0.854 F1=0.734  B: PR-AUC=0.870 F1=0.744
[BiLSTM_Full] epoch 9 train_loss=0.3737 val_loss=1.1094 A: PR-AUC=0.859 F1=0.739  B: PR-AUC=0.873 F1=0.749
[BiLSTM_Full] epoch 10 train_loss=0.3576 val_loss=1.1670 A: PR-AUC=0.861 F1=0.707  B: PR-AUC=0.879 F1=0.730
[BiLSTM_Full] epoch 11 train_loss=0.

In [20]:
# === Run GBM baseline (A1) ===
res_A1 = run_gbm()
print("GBM:", res_A1)

[LightGBM] [Info] Number of positive: 18464, number of negative: 10339
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017901 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16385
[LightGBM] [Info] Number of data points in the train set: 28803, number of used features: 101
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.641044 -> initscore=0.579900
[LightGBM] [Info] Start training from score 0.579900
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's average_precision: 0.948609	valid_0's binary_logloss: 0.340291
[LightGBM] [Info] Number of positive: 18327, number of negative: 10476
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016577 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16385
[LightGBM] [Info] Number of data points in the tr

In [21]:
# ================== Ablations (A2–A10) ==================
results=[dict(model="GBM (A1)",**res_A1), dict(model="Bi-LSTM (Full)",**met_full)]
order=["A2","A3","A5","A6","A7","A8","A9","A10","A4"]
name_map={"A2":"Bi-LSTM (No Context)","A3":"Bi-LSTM (No Gating)","A5":"Bi-LSTM (Lag)","A6":"Bi-LSTM (Short Seq)",
          "A7":"Bi-LSTM (Lag Robust)","A8":"Bi-LSTM (MapFree)","A9":"Bi-LSTM (Focal)","A10":"TCN Small","A4":"Bi-LSTM (Big)"}
for code in order:
    try:
        # TODO: insert your seq rebuilding logic here if features differ
        mdl = BiLSTM(X_train.shape[-1],S_train.shape[-1],hidden=CFG['HIDDEN'],gated=True) if code!="A10" else TCN(X_train.shape[-1],S_train.shape[-1],ch=CFG['TCN_CHANNELS'])
        mdl=train_model(mdl,code,"focal" if code=="A9" else "bce_weighted",dl_train,dl_val,yA_train,yB_train)
        pA,pB,pC=predict_model(mdl,ds_test); m=eval_all(code,yA_test,yB_test,yC_test,pA,pB,pC,test['meta'])
        results.append(dict(model=name_map[code],**m))
    except Exception as e:
        print(f"❌ {code} failed: {e}")
        results.append(dict(model=name_map[code],precision_auc_A=np.nan,f1_A=np.nan,precision_auc_B=np.nan,rmse_C=np.nan,
                            ece=np.nan,ttd_s=np.nan,fp_per_hour=np.nan))

# ================== Save Results ==================
df_res=pd.DataFrame(results)
excel_path, csv_path = OUT_DIR/"DBRA24_results.xlsx", OUT_DIR/"DBRA24_results.csv"
df_res.to_csv(csv_path,index=False); df_res.to_excel(excel_path,index=False)
print("Saved results to:",excel_path,csv_path)
display(df_res)

[A2] epoch 1 train_loss=0.4860 val_loss=1.2171 A: PR-AUC=0.821 F1=0.647  B: PR-AUC=0.831 F1=0.651
[A2] epoch 2 train_loss=0.4273 val_loss=1.1863 A: PR-AUC=0.833 F1=0.661  B: PR-AUC=0.846 F1=0.653
[A2] epoch 3 train_loss=0.4181 val_loss=1.2092 A: PR-AUC=0.835 F1=0.646  B: PR-AUC=0.846 F1=0.644
[A2] epoch 4 train_loss=0.4138 val_loss=1.1917 A: PR-AUC=0.837 F1=0.647  B: PR-AUC=0.851 F1=0.667
[A2] epoch 5 train_loss=0.4096 val_loss=1.1497 A: PR-AUC=0.840 F1=0.699  B: PR-AUC=0.853 F1=0.712
[A2] epoch 6 train_loss=0.4043 val_loss=1.1982 A: PR-AUC=0.841 F1=0.652  B: PR-AUC=0.856 F1=0.668
[A2] epoch 7 train_loss=0.3966 val_loss=1.1716 A: PR-AUC=0.847 F1=0.687  B: PR-AUC=0.863 F1=0.687
[A2] epoch 8 train_loss=0.3865 val_loss=1.1563 A: PR-AUC=0.851 F1=0.729  B: PR-AUC=0.867 F1=0.688
[A2] epoch 9 train_loss=0.3723 val_loss=1.1193 A: PR-AUC=0.853 F1=0.736  B: PR-AUC=0.874 F1=0.734
[A2] epoch 10 train_loss=0.3562 val_loss=1.1185 A: PR-AUC=0.865 F1=0.752  B: PR-AUC=0.883 F1=0.738
[A2] epoch 11 train

,model,precision_auc_A,f1_A,precision_auc_B,rmse_C,ece,ttd_s,fp_per_hour
0,GBM (A1),0.982549,0.958790,0.985423,0.033293,0.064044,0.161149,30.885329
1,Bi-LSTM (Full),0.947470,0.900970,0.944539,0.064036,0.050045,0.452822,68.752108
2,Bi-LSTM (No Context),0.954274,0.914124,0.954677,0.065887,0.041885,0.363186,61.618887
3,Bi-LSTM (No Gating),0.785460,0.839136,0.789807,0.061519,0.101538,0.000000,199.502530
4,Bi-LSTM (Lag),0.950832,0.908777,0.956069,0.063429,0.088647,0.530844,54.713322
5,Bi-LSTM (Short Seq),0.948769,0.903887,0.952976,0.064452,0.080681,0.416424,68.145025
6,Bi-LSTM (Lag Robust),0.959205,0.924437,0.962469,0.063893,0.035853,0.339799,52.133221
7,Bi-LSTM (MapFree),0.954255,0.913759,0.955402,0.064097,0.025148,0.456468,56.003373
8,Bi-LSTM (Focal),0.972366,0.950514,0.968369,0.060687,0.078379,0.230421,34.072513
9,TCN Small,0.863947,0.845501,0.862240,0.066423,0.114199,0.316465,150.480607


## 13) 📊 Export Results (Excel, CSV, Plots) + Final Comparison Table
<span style="background:#FFF8E1;color:#B37400;padding:6px 10px;border-radius:6px;display:inline-block;">
All artifacts are saved under <code>/kaggle/working/</code>.
</span>

In [22]:
# ===============================
# Results Saving (Clean Version)
# ===============================

import numpy as np
import pandas as pd

# Expected columns mapping (only those you *might* compute)
final_cols = {
    'model'        : 'Model / Ablation',
    'precision_auc_A': 'PR-AUC (A)',
    'f1_A'         : 'F1 (A)',
    'precision_auc_B': 'PR-AUC (B)',
    'rmse_C'       : 'RMSE (C)',
    'ece'          : 'ECE',
    'ttd_s'        : 'TTD (s) ↓',
    'fp_per_hour'  : 'FP/h ↓',
    'latency_ms'   : 'Latency (ms) ↓',
    'flops_m'      : 'FLOPs (M) ↓'
}

# ✅ Only keep the ones that exist in df_res
available_cols = {k:v for k,v in final_cols.items() if k in df_res.columns}

# Rename + reorder
df_final = df_res.rename(columns=available_cols)
df_final = df_final[[available_cols[k] for k in available_cols]]

# Save Excel + CSV
excel_path = OUT_DIR/"DBRA24_results.xlsx"
csv_path   = OUT_DIR/"DBRA24_results.csv"

with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    df_final.to_excel(writer, sheet_name="Summary", index=False)
    df_res.to_excel(writer, sheet_name="Raw", index=False)

df_final.to_csv(csv_path, index=False)

print(f"Saved Excel  -> {excel_path}")
print(f"Saved CSV    -> {csv_path}")

# Markdown table for paper
def to_md_table(df):
    hdr = "| " + " | ".join(df.columns) + " |"
    sep = "| " + " | ".join(["---"]*len(df.columns)) + " |"
    rows = ["| " + " | ".join(f"{x:.3g}" if isinstance(x,(int,float,np.floating)) else str(x) for x in r) + " |" for r in df.values]
    return "\n".join([hdr, sep] + rows)

print("\n### Final Comparison Table (ready to paste)\n")
print(to_md_table(df_final))


Saved Excel  -> out/DBRA24_results.xlsx
Saved CSV    -> out/DBRA24_results.csv

### Final Comparison Table (ready to paste)

| Model / Ablation | PR-AUC (A) | F1 (A) | PR-AUC (B) | RMSE (C) | ECE | TTD (s) ↓ | FP/h ↓ |
| --- | --- | --- | --- | --- | --- | --- | --- |
| GBM (A1) | 0.983 | 0.959 | 0.985 | 0.0333 | 0.064 | 0.161 | 30.9 |
| Bi-LSTM (Full) | 0.947 | 0.901 | 0.945 | 0.064 | 0.05 | 0.453 | 68.8 |
| Bi-LSTM (No Context) | 0.954 | 0.914 | 0.955 | 0.0659 | 0.0419 | 0.363 | 61.6 |
| Bi-LSTM (No Gating) | 0.785 | 0.839 | 0.79 | 0.0615 | 0.102 | 0 | 200 |
| Bi-LSTM (Lag) | 0.951 | 0.909 | 0.956 | 0.0634 | 0.0886 | 0.531 | 54.7 |
| Bi-LSTM (Short Seq) | 0.949 | 0.904 | 0.953 | 0.0645 | 0.0807 | 0.416 | 68.1 |
| Bi-LSTM (Lag Robust) | 0.959 | 0.924 | 0.962 | 0.0639 | 0.0359 | 0.34 | 52.1 |
| Bi-LSTM (MapFree) | 0.954 | 0.914 | 0.955 | 0.0641 | 0.0251 | 0.456 | 56 |
| Bi-LSTM (Focal) | 0.972 | 0.951 | 0.968 | 0.0607 | 0.0784 | 0.23 | 34.1 |
| TCN Small | 0.864 | 0.846 | 0.862 | 0.066

In [23]:
import shutil, os

# Path where Kaggle saves outputs
OUT_DIR = "/kaggle/working"

# Zip file name
zip_path = "/kaggle/working/output_results.zip"

# Remove old zip if exists
if os.path.exists(zip_path):
    os.remove(zip_path)

# Create new zip (recursively includes all files in working dir)
shutil.make_archive(zip_path.replace(".zip",""), 'zip', OUT_DIR)

print(f"✅ Zipped all outputs to {zip_path}")


✅ Zipped all outputs to /kaggle/working/output_results.zip
